In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import os
os.chdir('/content/drive/MyDrive/Projects/reddit-vote-predictor')
import sys
import pandas as pd
from datetime import datetime
# Import custom module
from scripts.RedditScrape import * # imports a class `RedditScrape`

In [4]:
# Reddit's own api:
!pip3 install psaw
#!pip3 install praw
import psaw
#import praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 28.3 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 


In [ ]:
# Set date range from which to extract data for two subreddits
# NOTE: ater running the data scraping and cleaning pipeline, one year of data
# resulted in around 1000 images, so we'll add time to the date range
# 10 years from the start of 2011 to the end of 2020
start_ = datetime(2011, 1, 1)
end_ = datetime(2020, 12, 31)
# Run the scraping pipeline using custom RedditScrape class imported above
cat = RedditScrape('cat', start_, end_)
cat.scrape_posts()
cat.posts.to_csv('data/tbl/cat_posts.csv', index=False)
dog = RedditScrape('dog', start_, end_)
dog.scrape_posts()
dog.posts.to_csv('data/tbl/dog_posts.csv', index=False)

In [5]:
# If the above is not run, read in the data
cat = pd.read_csv('data/tbl/cat_posts.csv', index_col = None)
dog = pd.read_csv('data/tbl/dog_posts.csv', index_col = None)